In [1]:
import numpy as np

class tree:
    def __init__(self, varNo, value, operator):
        self.rootNode = treeNode(0, value, varNo=varNo, operator=operator) # index 0 because it is the root node
        self.nodes = []
        self.nodes.append(self.rootNode)
        self.leafNodes = []
        self.leafNodes.append(0) # initialize leaf node list for later analysis
    
    def addNode(self, ChildOf, branch, value, operator='<', varNo=0):
        node = treeNode(len(self.nodes),value,ChildOf=ChildOf,operator=operator,varNo=varNo)
        self.leafNodes.append(node.number)
        self.nodes.append(node)
        
        parent = self.nodes[ChildOf]
        if branch:
            parent.leftTrue = node
        else: 
            parent.rightFalse = node
            
        if parent.leftTrue is not None and parent.rightFalse is not None: # parent is no longer a leaf node
            toDelete = self.leafNodes.index(parent.number)
            del self.leafNodes[toDelete]
        
        return(node.number)
    
    
    def trace(self, x):
        traceRoute = self.rootNode.trace(x)[0] # traverse the tree by orientating along route x for respective children
        return traceRoute # list of lists which display which nodes were traversed for a given feature vector (last node is corresponding leaf)
    
    def eval(self, x):
        traceRoute = self.trace(x)
        y = np.zeros(len(traceRoute))
        for i in range(len(y)):
            # write down last element of corresponding traceRoutes, and access the associated node value to write in return vector
            y[i] = self.nodes[traceRoute[i][-1]]()
        return(y)
    

In [9]:
class treeNode:
    def __init__(self, number, value, ChildOf=None, operator='<', varNo=0):
        self.number = number
        self.childOf = ChildOf
        self.leftTrue = None
        self.rightFalse = None
        self.value = value
        self.varNo = varNo
        self.operator = operator
    
    def __call__(self):
        return(self.value)
    
    def leafNode(self):
        if self.leftTrue is not None and self.rightFalse is not None:
            return(False)
        else:
            return True
    
    def evalCondition(self, x):
        # evalute given condition (speficified as string in operator attribute) on feature x
        if self.operator == '=':
            # for all samples, filter out variable at given position and evaluate according to operation
            cond = x[:, self.varNo] == self.value
        elif self.operator == '<':
            cond = x[:, self.varNo] < self.value
        else: #case >
            cond = x[:, self.varNo] > self.value
        return cond
    
    def trace(self, x, index=None, traceRoute=None):
        if index is None:
            # match feature vectors (samples) to its position in the sample collection
            index = np.arange(len(x))
        if traceRoute is None:
            # initiate a corresponding empty list for each of the given features in one sample (for all samples)
            traceRoute = [[] for x in range(len(x))]
            
        # for every given sample 
        for k in index:
            # append our number to the traceRoute for a given sample
            traceRoute[k].append(self.number)

        if self.leafNode():
            return (traceRoute, index) # if we arrive at a leaf the corresponding route is finished

        cond = self.evalCondition(x[index]) # evalute the given condition according to the node operator for all indexed samples
        
        trueIndex = index[cond] # stack of feature vectors for which the evaluation was positive (used as a filter for next trace)
        falseIndex = index[~cond] # stack of feature vectors for which the evaluation was negative (used as a filter for next trace)

        if self.leftTrue is not None and trueIndex.size != 0: # if there is a left child and we have at least one route that leads there
            traceRoute = self.leftTrue.trace(x, trueIndex, traceRoute)[0] # continue trace at left node with filtered index and existing route (filter traceRoute from return value)
        if self.rightFalse is not None and falseIndex.size != 0: # same for right
            traceRoute = self.rightFalse.trace(x, falseIndex, traceRoute)[0]
        return (traceRoute, index)


In [11]:
# manually build a simple (pure) decision tree structure
bicycleTree = tree(0,1,'=')
No = bicycleTree.addNode(0,False,1,varNo=1,operator='=')
bicycleTree.addNode(No, False, 0)
bicycleTree.addNode(No, True, 1)
No = bicycleTree.addNode(0, True, 1, varNo=2, operator='=')
bicycleTree.addNode(No, True, 0)
No = bicycleTree.addNode(No, False, 1, varNo=3, operator='=')
bicycleTree.addNode(No, True, 0)
bicycleTree.addNode(No, False, 1)

# build a sample
x = np.array([True, False, False, False]).reshape(1,4)
# evaluate and trace
y = bicycleTree.eval(x)
traceRoute = bicycleTree.trace(x)

In [12]:
traceRoute # traversal of node 0, 4, 6 and arrival at leaf 8

[[0, 4, 6, 8]]

In [14]:
# stack samples to test parallel execution
x = np.random.randint(2, size=(10, 4))
# evaluate and trace
y = bicycleTree.eval(x)
traceRoute = bicycleTree.trace(x)

In [15]:
# Start with automatic tree generation using the Iterative Dichotomiser 3 algorithm


[[0, 1, 3],
 [0, 4, 5],
 [0, 4, 6, 8],
 [0, 4, 6, 8],
 [0, 1, 2],
 [0, 4, 5],
 [0, 4, 6, 8],
 [0, 4, 6, 8],
 [0, 4, 5],
 [0, 4, 6, 8]]

In [2]:
import numpy as np

def weightedSelfInformation(x):
    y = 0 if x <= 0 else x*np.log2(x)
    return (y)

In [6]:
def calConditionalEntropy(y,D,Feature):
    sizeDataBase = D.shape[0]
    D = D.astype(bool)
    TrueFeatureDatabase = np.sum(D[:,Feature])
    FalseFeatureDatabase = sizeDataBase - TrueFeatureDatabase
    PFeatureTrue = TrueFeatureDatabase/sizeDataBase
    PFeatureFalse = FalseFeatureDatabase/sizeDataBase
    
    Htrue = 0
    if PFeatureTrue>0:
        P_AB_True = TrueFeatureDatabase - np.sum(np.logical_and(D[:,Feature],y))
        P_AB_False = TrueFeatureDatabase - P_AB_True
        P_AB_True = P_AB_True/TrueFeatureDatabase
        P_AB_False = P_AB_False/TrueFeatureDatabase
        Htrue = PFeatureTrue * (weightedSelfInformation(P_AB_False) + weightedSelfInformation(P_AB_True))
    Hfalse = 0
    if PFeatureFalse == P_AB_False/FalseFeatureDatabase:
        P_AB_True = FalseFeatureDatabase - np-sum(np.logical_and(~D[:,Feature], y))
        P_AB_False = FalseFeatureDatabase - P_AB_True
        P_AB_True = P_AB_True/FalseFeatureDatabase
        P_AB_False = P_AB_False/FalseFeatureDatabase
        Hfalse = PFeatureFalse * (weightedSelfInformation(P_AB_False) + weightedSelfInformation(P_AB_True))
    
    H = -Htrue - Hfalse
    return H